In [1]:
import zquery
import zquery.analysis_functions
import numpy as np
import pandas as pd
import glob
import yaml
import functools

In [2]:
%load_ext line_profiler
%load_ext memory_profiler

In [3]:
with open("configs/mc.yaml", 'r') as f:
    cfg = yaml.safe_load(f)
    
with open("configs/mc_test.yaml", 'r') as f:
    cfg_test = yaml.safe_load(f)

In [4]:
paths = sorted(glob.glob("/vols/cms/sdb15/Analysis/ZinvWidth/databases/full/2020/02_Feb/10_SingleTable_FixObjectWeights/new_partitions/mc/*.h5"))
paths = paths[:5]

In [5]:
%lprun -f zquery.analysis_functions.histogram zquery.process_tables(cfg, ["histogram"], paths)

Timer unit: 1e-06 s

Total time: 59.9949 s
File: /vols/build/cms/sdb15/phd/ZinvWidth/zinv-notebooks/externals/zquery/zquery/analysis_functions.py
Function: histogram at line 366

Line #      Hits         Time  Per Hit   % Time  Line Contents
   366                                           def histogram(path, **kwargs):
   367         5       8819.0   1763.8      0.0      df_hist = pd.DataFrame()
   368        10   26795713.0 2679571.3     44.7      for df in pd.read_hdf(path, **kwargs["input"]):
   369         5         24.0      4.8      0.0          if "lambdas" in kwargs:
   370        15         60.0      4.0      0.0              for key, func in kwargs["lambdas"].items():
   371        10    1997984.0 199798.4      3.3                  df[key] = eval(f'lambda {func}')(df)
   372                                           
   373         5          6.0      1.2      0.0          if "evals" in kwargs:
   374         5    1344756.0 268951.2      2.2              df.eval("\n".join(kw

In [6]:
%lprun -f zquery.analysis_functions.histogram zquery.process_tables(cfg_test, ["histogram"], paths)

Timer unit: 1e-06 s

Total time: 62.3265 s
File: /vols/build/cms/sdb15/phd/ZinvWidth/zinv-notebooks/externals/zquery/zquery/analysis_functions.py
Function: histogram at line 366

Line #      Hits         Time  Per Hit   % Time  Line Contents
   366                                           def histogram(path, **kwargs):
   367         5       5042.0   1008.4      0.0      df_hist = pd.DataFrame()
   368        10   26881458.0 2688145.8     43.1      for df in pd.read_hdf(path, **kwargs["input"]):
   369         5         23.0      4.6      0.0          if "lambdas" in kwargs:
   370        15         55.0      3.7      0.0              for key, func in kwargs["lambdas"].items():
   371        10    1944053.0 194405.3      3.1                  df[key] = eval(f'lambda {func}')(df)
   372                                           
   373         5          7.0      1.4      0.0          if "evals" in kwargs:
   374         5    1133478.0 226695.6      1.8              df.eval("\n".join(kw

In [7]:
%mprun -f zquery.analysis_functions.histogram zquery.process_tables(cfg, ["histogram"], paths)

Filename: /vols/build/cms/sdb15/phd/ZinvWidth/zinv-notebooks/externals/zquery/zquery/analysis_functions.py

Line #    Mem usage    Increment   Line Contents
   366    225.9 MiB    218.5 MiB   def histogram(path, **kwargs):
   367    225.9 MiB      0.0 MiB       df_hist = pd.DataFrame()
   368   1842.5 MiB   1368.7 MiB       for df in pd.read_hdf(path, **kwargs["input"]):
   369   1588.1 MiB      0.0 MiB           if "lambdas" in kwargs:
   370   1714.2 MiB      0.0 MiB               for key, func in kwargs["lambdas"].items():
   371   1714.2 MiB    126.9 MiB                   df[key] = eval(f'lambda {func}')(df)
   372                             
   373   1714.2 MiB      0.0 MiB           if "evals" in kwargs:
   374   1767.6 MiB     53.4 MiB               df.eval("\n".join(kwargs["evals"]), inplace=True)
   375                             
   376   1842.5 MiB      0.0 MiB           for cfg in kwargs["cfgs"]:
   377   1842.5 MiB      0.0 MiB               for keymap in cfg["eval_keys"

In [8]:
%mprun -f zquery.analysis_functions.histogram zquery.process_tables(cfg_test, ["histogram"], paths)

Filename: /vols/build/cms/sdb15/phd/ZinvWidth/zinv-notebooks/externals/zquery/zquery/analysis_functions.py

Line #    Mem usage    Increment   Line Contents
   366    258.7 MiB    223.7 MiB   def histogram(path, **kwargs):
   367    258.7 MiB      0.0 MiB       df_hist = pd.DataFrame()
   368   1845.6 MiB   1363.2 MiB       for df in pd.read_hdf(path, **kwargs["input"]):
   369   1609.4 MiB      0.0 MiB           if "lambdas" in kwargs:
   370   1721.1 MiB      0.0 MiB               for key, func in kwargs["lambdas"].items():
   371   1721.1 MiB    126.2 MiB                   df[key] = eval(f'lambda {func}')(df)
   372                             
   373   1721.1 MiB      0.0 MiB           if "evals" in kwargs:
   374   1771.9 MiB     51.6 MiB               df.eval("\n".join(kwargs["evals"]), inplace=True)
   375                             
   376   1845.6 MiB      0.0 MiB           for cfg in kwargs["cfgs"]:
   377   1845.6 MiB      0.0 MiB               for keymap in cfg["eval_keys"

In [1]:
result = functools.reduce(lambda x,y: zquery.analysis_functions._df_merge(x,y), [r for rs in results for r in rs])
result.to_hdf(
    "data/dists.h5", "MCAggEvents", 
    format='table', append=False,
    complib='zlib', complevel=9,
)

NameError: name 'functools' is not defined